# Few-Shot Prompts

This tutorial shows how to build few-shot prompts using dynamic format specs and list interpolation.

Few-shot prompts provide examples to guide model behavior. With t-prompts, you can:
- Generate keys programmatically (e.g., `example_0`, `example_1`)
- Interpolate lists of examples with custom separators
- Navigate and inspect the structured examples

In [ ]:
from t_prompts import dedent, prompt

## Dynamic Format Specs

Format specs can contain interpolated expressions just like f-strings. This lets you create keys programmatically.

In [ ]:
# Use interpolated values in format specs to create dynamic keys
task_type = "translation"
field_name = "user_input"
user_message = "Translate this to French"

# The format spec {task_type}_{field_name} evaluates to "translation_user_input"
p = prompt(t"{user_message: {task_type}_{field_name} }")

print(f"Rendered text: {str(p)}")
print(f"Generated key: {list(p.keys())[0]}")
print(f"Value: {p['translation_user_input']}")

## Creating Numbered Example Keys

Dynamic format specs are perfect for creating numbered examples like `example_0`, `example_1`, etc.

In [ ]:
# Sample sentences for examples
sentences = [
    "The cat sat on the mat.",
    "Python is a programming language.",
    "Machine learning is fascinating."
]

# Create prompts with numbered keys: example_0, example_1, example_2
example_prompts = []
for i, sentence in enumerate(sentences):
    # Convert i to string since only strings can be interpolated
    p = prompt(t"{sentence: example_{str(i)} }")
    example_prompts.append(p)
    print(f"Created key: {list(p.keys())[0]}")

# Verify we created 3 prompts
print(f"\nTotal examples: {len(example_prompts)}")

## List Interpolation with Default Separator

You can interpolate a list of `StructuredPrompt` objects. By default, they're joined with newlines.

In [ ]:
# Interpolate the list of example prompts
few_shot = prompt(t"Examples:\n{example_prompts:examples}")

print("Rendered output (default newline separator):")
print(str(few_shot))
print(f"\nThe 'examples' key contains {len(few_shot['examples'])} items")

## Custom Separators

Use `sep=<value>` in the format spec to specify a custom separator between list items.

In [ ]:
# Create fresh examples for different separator tests
examples_for_pipe = [
    prompt(t"{sentences[i]: ex_{str(i)} }")
    for i in range(len(sentences))
]

examples_for_comma = [
    prompt(t"{sentences[i]: ex_{str(i)} }")
    for i in range(len(sentences))
]

# Pipe separator
with_pipes = prompt(t"Examples: {examples_for_pipe:examples:sep= | }")
print("With ' | ' separator:")
print(str(with_pipes))

# Comma separator
with_commas = prompt(t"Examples: {examples_for_comma:examples:sep=, }")
print("\nWith ', ' separator:")
print(str(with_commas))

## Navigating List Items

You can navigate into list interpolations and access individual items by index.

In [ ]:
# Access the list node
examples_list = few_shot['examples']
print(f"List contains {len(examples_list)} items")

# Access the first item in the list
first_item = examples_list[0]
print(f"\nFirst item type: {type(first_item).__name__}")
print(f"First item key: {list(first_item.keys())[0]}")
print(f"First item value: {first_item['example_0']}")

# Access the second item
second_item = examples_list[1]
print(f"\nSecond item key: {list(second_item.keys())[0]}")
print(f"Second item value: {second_item['example_1']}")

## Complete Few-Shot Prompt Example

Let's build a realistic few-shot prompt for a question-answering task.

In [ ]:
# Question-answer pairs for few-shot examples
qa_pairs = [
    ("What is 2+2?", "4"),
    ("What is the capital of France?", "Paris"),
    ("What is Python?", "A programming language"),
]

# Build structured examples with Q: and A: format
structured_examples = []
for i, (question, answer) in enumerate(qa_pairs):
    # Each example has two interpolations: question and answer
    ex = prompt(t"Q: {question: q_{str(i)} }\nA: {answer: a_{str(i)} }")
    structured_examples.append(ex)

# Build the complete few-shot prompt
instruction = "Answer the following question based on these examples:"
new_question = "What is 5+3?"

full_prompt = prompt(
    t"{instruction:instruction}\n\n{structured_examples:examples:sep=\n\n}\n\nQ: {new_question:new_q}\nA:"
)

print("Complete few-shot prompt:")
print(str(full_prompt))

# Inspect the structure
print("\n--- Structure ---")
print(f"Top-level keys: {list(full_prompt.keys())}")
print(f"Number of examples: {len(full_prompt['examples'])}")
print(f"First example keys: {list(full_prompt['examples'][0].keys())}")

## Advanced: Mixing Static and Dynamic Keys

You can combine static keys with dynamically generated keys in the same prompt.

In [ ]:
# System prompt with static key
system_msg = "You are a helpful math tutor."

# Dynamic examples
math_examples = [
    ("10 + 15", "25"),
    ("7 * 8", "56"),
]

example_list = [
    prompt(t"Problem: {prob: prob_{str(i)} } = {ans: ans_{str(i)} }")
    for i, (prob, ans) in enumerate(math_examples)
]

# User query with static key
user_problem = "12 * 12"

# Combine everything
final_prompt = dedent(t"""
    {system_msg:system}

    Here are some examples:
    {example_list:examples:sep=\n}

    Now solve:
    Problem: {user_problem:user_input} =
    """)

print(str(final_prompt))

# Show the mixed key structure
print(f"\nAll top-level keys: {list(final_prompt.keys())}")
print("Static keys: system, user_input")
print(f"Dynamic keys in examples: {list(final_prompt['examples'][0].keys())}")

## Summary

You've learned how to:
- Use dynamic format specs to generate keys programmatically
- Create numbered examples with `{value: key_{str(i)} }`
- Interpolate lists of prompts with custom separators
- Navigate into list structures to access individual items
- Build complete few-shot prompts for real-world tasks

This pattern is essential for building flexible, reusable prompt templates where the number and content of examples can vary.